# Modeling

In [1]:
import ppdet
# ppdet.model_zoo.list_model(['picodet'])
# ppdet.model_zoo.list_model(['ssd'])


In [20]:
model = ppdet.model_zoo.get_model('picodet\picodet_s_320_coco_lcnet', pretrained=False)


In [24]:
model_ssd = ppdet.model_zoo.get_model('ssd\ssd_mobilenet_v1_300_120e_voc', pretrained=False)

In [2]:
model_custom = ppdet.model_zoo.get_model('picodet\picodet_s_300_custom', pretrained=False)

# Below for QAT(ACT)

In [13]:
# export
config_yml = r"configs\ssd\ssdlite_mobilenet_v3_small_custom.yml"
config_yml = r"configs\ssd\ssd_mobilenet_v1_custom.yml"
config_yml = r"configs/picodet/picodet_s_300_custom.yml"
weight_path = r"..\ssdlite_models\before_convert\best_model.pdparams"
weight_path = r"..\ssd_models\before_convert\best_model.pdparams"
weight_path = r"output/picodet_s_300_custom/best_model.pdparams"
!python tools/export_model.py -c $config_yml -o weights=$weight_path

[04/06 18:10:23] ppdet.utils.checkpoint INFO: Finish loading model weights: output/picodet_s_300_custom/best_model.pdparams
[04/06 18:10:23] ppdet.engine INFO: Export inference config file to output_inference\picodet_s_300_custom\infer_cfg.yml
[04/06 18:10:26] ppdet.engine INFO: Export model and saved in output_inference\picodet_s_300_custom


In [ ]:
act_yml = "ssdlite_qat.yml"
!python deploy/auto_compression/run.py --config_path=$act_yml --save_dir='./output/ssd_lite_qat_distilled'  

# Test

In [4]:
import numpy as np
picodet_weight = np.fromfile(r"F:\CICC\env\PaddleDetection\output\picodet_s_288_custom_trained_input-fp32\model_final.pdparams")
ssd_weight = np.fromfile(r"F:\CICC\env\ssd_models\after_convert\ssd_mobilenet_v1_custom\model.pdiparams")

In [11]:
import paddle
paddle.enable_static()
exe = paddle.static.Executor(paddle.CPUPlace())
[inference_program, feed_target_names, fetch_targets] = (paddle.static.load_inference_model(r"F:\CICC\env\picodet_models\after_convert\picodet_s_300_custom_fp-norm\model", exe))

# View Sensitivity Data


In [3]:
import pickle
# data = pickle.load(open("sensitivity_0.data", "rb"))
data = pickle.load(open("ssd_sensitivity.pkl", "rb"))
print(data)

{'conv2d_0.w_0': {0.1: 0.0, 0.2: 0.0, 0.3: 0.003500627556567802, 0.4: 0.2971673592612734, 0.5: 0.4334470306447954, 0.6: 0.7746163277826341, 0.7: 0.7351857343574354, 0.8: 0.8203384240375886, 0.9: 0.9615516054076922}, 'conv2d_2.w_0': {0.1: -0.002567325424914282, 0.2: 0.046226160468661834, 0.3: 0.08960231133679743, 0.4: 0.49531454727381696, 0.5: 0.543520782846043, 0.6: 0.5679049127193854, 0.7: 0.7003999336647315, 0.8: 0.7514330167185593, 0.9: 0.9975155297761068}, 'conv2d_4.w_0': {0.1: 0.14887707306975625, 0.2: 0.19924686159814645, 0.3: 0.48529603873071336, 0.4: 0.3223545910088867, 0.5: 0.6017425979255845, 0.6: 0.9589102673054066, 0.7: 1.0, 0.8: 0.9996273294664161, 0.9: 0.9997133303587815}, 'conv2d_6.w_0': {0.1: 0.06985842334798163, 0.2: 0.3028273536066298, 0.3: 0.5245441039404551, 0.4: 0.6781175334581678, 0.5: 0.8416711851033856, 0.6: 0.9543578695814701, 0.7: 0.9899539578616249, 0.8: 0.9979975291711318, 0.9: 0.9941705179932009}, 'conv2d_8.w_0': {0.1: 0.07695905933476488, 0.2: 0.1495362504

In [6]:
from paddleslim.prune import get_ratios_by_loss

pruned_ratio = get_ratios_by_loss(data, 0.2)
ratio = list(pruned_ratio.values())